In [21]:
# Import packages
import numpy as np
import cvxpy as cp
import time
import math
import phi_divergence as phi
import robust_sampling as rs
import dataio
import util as util
import scipy.stats

The problem we examine is as follows:

\begin{align}
\label{math_form:examples:pm2}
    \max_{\mathbf{x}}&~\theta \\
    \text{s.t.}&~\mathbf{r}^T \mathbf{x} \geq \theta \\
    &~\mathbf{e}^T \mathbf{x} = 1, \\
    &~\mathbf{x} \geq 0,
\end{align}

where $\mathbf{x}, \mathbf{r} \in \mathbb{R}^{k}$

We randomly sample $N$ returns for k assets, which is done in the following way:

\begin{equation}
\tilde{r}_{i}=\left\{\begin{array}{ll}
\frac{\sqrt{\left(1-\gamma_{i}\right) \gamma_{i}}}{\gamma_{i}} & \text { with probability } \gamma_{i} \\[2mm]
-\frac{\sqrt{\left(1-\gamma_{i}\right) \gamma_{i}}}{1-\gamma_{i}} & \text { with probability } 1-\gamma_{i}
\end{array}, \quad \gamma_{i}=\frac{1}{2}\left(1+\frac{i}{k + 1}\right), \quad i=1, \ldots, k. \right.
\end{equation}

In [2]:
# Problem specific functions:
def generate_data(k, N):
    np.random.seed(1)
    gamma = np.fromiter(((1/2)*(1 + (i/(k+1))) for i in range(1,k+1)), float)
    return_pos = np.fromiter(((math.sqrt((1-gamma[i])*gamma[i])/gamma[i]) for i in range(0,k)), float)
    return_neg = np.fromiter((-(math.sqrt((1-gamma[i])*gamma[i])/(1-gamma[i])) for i in range(0,k)), float)
    returns = np.empty([N,k])
    for n in range(0, N):
        for i in range(0, k):
            prob = np.random.uniform()
            if prob <= gamma[i]:
                returns[n, i] = return_pos[i]
            else:
                returns[n, i] = return_neg[i]
    return returns 

def solve_SCP(S, time_limit):
    k = S.shape[1]
    x = cp.Variable(k, nonneg = True)
    theta = cp.Variable(1)
    constraints = [theta - (S @ x) <= 0, cp.sum(x) == 1]
    obj = cp.Maximize(theta)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK, mosek_params = {mosek.dparam.optimizer_max_time: time_limit})
    x_value = np.concatenate((theta.value,x.value)) # Combine x and theta into 1 single solution vector
    return(x_value, prob.value)

def uncertain_constraint(S, x):
    return x[0] - (S @ x[1:]) # Assume that x[0] contains theta variable 

In [12]:
k = 10
N = 100

In [19]:
returns = generate_data(k, N)
returns

array([[ 0.91287093, -1.20185043,  0.75592895,  0.68313005,  0.61237244,
         0.54232614,  0.47140452,  0.39735971,  0.31622777,  0.21821789],
       [ 0.91287093, -1.20185043,  0.75592895, -1.46385011,  0.61237244,
         0.54232614,  0.47140452,  0.39735971,  0.31622777,  0.21821789],
       [-1.09544512, -1.20185043,  0.75592895, -1.46385011, -1.63299316,
        -1.84390889,  0.47140452,  0.39735971,  0.31622777,  0.21821789],
       [ 0.91287093,  0.83205029, -1.32287566,  0.68313005,  0.61237244,
         0.54232614,  0.47140452,  0.39735971,  0.31622777,  0.21821789],
       [-1.09544512, -1.20185043,  0.75592895, -1.46385011,  0.61237244,
         0.54232614, -2.12132034,  0.39735971,  0.31622777,  0.21821789],
       [ 0.91287093, -1.20185043,  0.75592895,  0.68313005,  0.61237244,
         0.54232614,  0.47140452,  0.39735971,  0.31622777,  0.21821789],
       [ 0.91287093,  0.83205029, -1.32287566,  0.68313005,  0.61237244,
         0.54232614,  0.47140452,  0.39735971

In [10]:
returns.shape[1]

5

In [41]:
x, obj = solve_SCP(returns, 60)

In [42]:
x

array([3.01511345e-01, 7.34484430e-13, 1.45647885e-11, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00])

In [44]:
x[0] - returns @ x[1:]

array([ 2.65815148e-12, -2.82384671e-11,  2.65815148e-12, -2.67486588e-11,
        4.14795975e-12, -2.67486588e-11, -2.82384671e-11,  2.65815148e-12,
        4.14795975e-12,  2.65815148e-12])

In [13]:
### Here we calculate the a-priori phi-divergence RO solution assuming no prior knowledge of the distribution of the returns
### This is the phi_dot for modified chi squared.
phi_dot = 2
phi_conj = phi.mod_chi2_conj
r = phi_dot/(2*N)*scipy.stats.chi2.ppf(0.1, N-1)
p = np.zeros(N)+1/N

In [25]:
import importlib
importlib.reload(util)

<module 'util' from 'C:\\Users\\gjin\\SamplingRobust\\Code\\util.py'>

In [26]:
beta = 0.95
print(util.af_RC_exp_pmin(p,returns,r,phi_conj,np.array([1/(1-beta),0]),np.array([0,1])))

(array([1.82734762e-01, 1.36936640e-07, 1.76538127e-01, 6.13294171e-07,
       1.63439884e-01, 1.53794999e-01, 1.41545503e-01, 4.56975947e-07,
       1.05499423e-01, 7.64453785e-02]), -0.5004342024795069)
